<a href="https://colab.research.google.com/github/Davilirio/Python_data_analysis/blob/master/drivers_license_info_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## American Driver's License information extractor

### Initial extractor with Regular Expressions:

In [ ]:
# Importing the libraries:
import pandas as pd 
import spacy
import re
import itertools
import nltk
from nameparser.parser import HumanName
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Lendo o arquivo:
# dataset = pd.read_csv('/home/davi/projetos/Access/driver_license_dataset/results1.csv', names = ['arquivo', 'conteudo'])
!python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=dc424c5fdb2f91614935406aaca3bf8552949b910d23a0ea02686ccf4e260831
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2zkc3d2/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
# Criando listas de arquivos e conteudo:

filenames = [str(x).lower() for x in list(dataset['arquivo'].values)]
carteiras = [str(x).lower() for x in list(dataset['conteudo'].values)]

In [ ]:
# Create a dict where the key is the file name, the values are dicts where 
# the keys are the variables to be found in the driver's license and the 
# values to those keys are the values found by the extractor
carteira = {}

for i in range(len(carteiras)):
    #st = sts.match(str(carteiras[i]))
    st = re.search(r' ?(georgia)|(utah)|(florida)|(orida)|(lorida) ?', str(carteiras[i]))
    s = re.search(r'((?<=sex )([f|m]))', str(carteiras[i]))
    o = re.search(r'((?<=eyes )([a-z]{1,3}))', str(carteiras[i]))
    iss = re.search(r'(?<=[iss |iss dd ])(\d{1,2}/\d{1,2}/\d{1,4})', str(carteiras[i]))
    dl = re.search(r'(?<=[dl |dln |4d ])(\d{9})', str(carteiras[i]))
    n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteiras[i]))
    ln = re.search(r'(?<=[ln | last name |1 ] )(\w{2,10})', str(carteiras[i]))
    fn = re.search(r'(?<=[fn | first name |2 ] )(\w{2,10})', str(carteiras[i]))
    # Setting the minimum of attributes found to three (3):
    num = ( bool( s ) + bool( o ) + bool( n ) + bool( iss ) + bool( dl )  )
    # If there are three (3) or more attributes found, the drivers license will be added:
    if num >= 2:
        carteira[filenames[i]] = dict(Sex =  s.group() if not s is None else '',
                           Eyes =  o.group() if not o is None else '',
                           Issued =  iss.group() if not iss is None else '',
                           DL_number =  dl.group() if not dl is None else '',
                            DOB =  n.group() if not n is None else '',
                                     last_name = ln.group() if not ln is None else '',
                                      first_name = fn.group() if not fn is None else '',
                                      state = st.group() if not st is None else '',
                                      text = carteiras[i]
                                     )

### Viewing the first 10 driver's licenses extracted:

In [ ]:
# Visualizando as primeiras 10 carteiras:
dict(itertools.islice(carteira.items(), 120))

### Division between the 3 main states attended and an outlier group:

In [ ]:
# !!!  Test area  !!!

t = 'florida the sunshine state driver license class e s530-003-67-834-0 andrea coney od we sneed 2400 nw 132nd st miami, fl 33167-1351 dob: 09-14-1967 sex: f issued: 10-12-2012 hgt: 5-02 expires: 09-14-2020 seal oft rest endorse: safe driver operation of a motor vehicle constitutes consent to any sobriety test required by law. '
m_teste =  re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', t)
n_teste = re.search(r' ?f?l?(orid)a? ?', t)
o_teste = re.search(r'(?<=[dl no. ])(\d{6,12})', t)
print(m_teste)

In [ ]:
# Divide and count how many from each of the 3 states:

florida_dl = dict()
utah_dl = dict()
georgia_dl = dict()
outliers = dict()

for i, carteira in enumerate(carteiras):
    florida_re = re.search(r' ?f?l?(orid)a? ?', str(carteira))
    dln_florida_re = re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', str(carteira))
    georgia_re = re.search(r' ?g?(eorgi)a? ?', str(carteira))
    dln_georgia_re = re.search(r'(?<=[dl no. ])(\d{6,12})', str(carteira))
    utah_re = re.search(r' ?(utah) ?', str(carteira))
    if florida_re or dln_florida_re:
        florida_dl[filenames[carteiras.index(carteira)]] = carteira
    if georgia_re:
        georgia_dl[filenames[carteiras.index(carteira)]] = carteira
    if utah_re:
        utah_dl[filenames[carteiras.index(carteira)]] = carteira
    if bool(florida_re) + bool(dln_florida_re) + bool(georgia_re) + bool(utah_re) == 0:
        outliers[filenames[carteiras.index(carteira)]] = carteira
        
        
print("The number of driver's licenses found for Florida, Utah and Georgia, respectively, are:  {}".format([len(florida_dl), len(utah_dl), len(georgia_dl)]))

### Defining for one Driver's License

In [ ]:

def state_classifier(carteira):
        florida_re = re.search(r' ?f?l?(orid)a? ?', str(carteira))
        dln_florida_re = re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', str(carteira))
        georgia_re = re.search(r' ?g?(eorgi)a? ?', str(carteira))
        dln_georgia_re = re.search(r'(?<=[dl no. ])(\d{6,12})', str(carteira))
        utah_re = re.search(r' ?(utah) ?', str(carteira))
        if florida_re or dln_florida_re:
            state = 'florida'
        if georgia_re:
            state = 'georgia'
        if utah_re:
            state = 'utah'
        if bool(florida_re) + bool(dln_florida_re) + bool(georgia_re) + bool(utah_re) == 0:
             state = 'outlier'
        
        return state

### Measuring informations obtained by the basic extractor:

In [ ]:
((len(utah_dl)+len(georgia_dl)+len(florida_dl))/len(carteiras))*100

## Personalized Extractors per State

### Florida:

In [ ]:
florida_dl_dict = dict()
for i in range(len(carteiras)):
    s = re.search(r'((?<=sex )([f|m]))', str(carteiras[i]))
    o = re.search(r'((?<=eyes )([a-z]{1,3}))', str(carteiras[i]))
    exp = re.search(r'(?<=[iss|iss dd ])(\d{1,2}/\d{1,2}/\d{1,4})', str(carteiras[i]))
    dl = re.search(r'(?<=[dln | 4d ])(\D{1}-\d{3}-\d{1,2}-\d{1-3}-\d{1})', str(carteiras[i]))
    n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteiras[i]))
    ln = re.search(r'(?<=[ln | last name ] )(\w{2,10})', str(carteiras[i]))
    fn = re.search(r'(?<=[fn | first name ] )(\w{2,10})', str(carteiras[i]))
    cl = re.search(r'(?<=[9 |class ])([a|b|c|e|])', str(carteiras[i]))
    #Setando o número mínimo de argumentos aceitos para 3:
    num = ( bool( s ) + bool( o ) + bool( n ) + bool( exp ) + bool( dl )  )
    #Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
    if num >= 2:
        florida_dl_dict[filenames[i]] = dict(Sexo =  s.group() if not s is None else '',
                            Olho =  o.group() if not o is None else '',
                            Expedicao =  exp.group() if not exp is None else '',
                           Numero_Carteira =  dl.group() if not dl is None else '',
                            Nascimento =  n.group() if not n is None else '',
                                     sobrenome = ln.group() if not ln is None else '',
                                      nome = fn.group() if not fn is None else '',
                                      classe = cl.group() if not cl is None else '')

In [ ]:
def florida_extractor(carteira):
    florida_dl_info = dict()
    s = re.search(r'((?<=sex )([f|m]))', str(carteira))
    o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(carteira))
    exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    dl = re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', str(carteira))
    n = re.search(r'(?<=[(dob )|(date of birth )] )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    cl = re.search(r'(?<=[9 |class ])([a|b|c|e|])', str(carteira))
    #Setando o número mínimo de argumentos aceitos para 3:
    num = ( bool( s ) + bool( o ) + bool( n ) + bool( exp ) + bool( dl )  )
    #Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
    if num >= 2:
        florida_dl_info[carteira] = dict(sex =  s.group() if not s is None else '',
                            eyes =  o.group() if not o is None else '',
                            expiration_date =  exp.group() if not exp is None else '',
                           dl_number =  dl.group() if not dl is None else '',
                            birth_date =  n.group() if not n is None else '',
                                        #  name = name_extractor(carteira),
                                        #  location = location_extractor(carteira),
                                      dl_class = cl.group() if not cl is None else '')
    
    return florida_dl_info 

In [ ]:
carteira_florida = "p. florida usa grigat .. driver license 9 class e *o dilm230-073-83-470-0 40dln macedo 2bruno miguel de pina a1339 sand lime rd winter garden, fl 34787-4291 3dob 12/30/1983 15 sex m 4b exp 12/30/202016 hgt 5-11\" 12 rest none si safe driver 31d0b suns staf t83 9a end none 1de hesunghi a5/eive arriv tate 4a iss 12/12/2012 ine 5 dd g711906070623 oooinoooo arrivealive replaced 06/07/2019 suneh veteran m he state operation of a motor vehicle constitutes consent to any sobriety test required by law. donor 10000000 tri "
florida_extractor(carteira_florida)

{'p. florida usa grigat .. driver license 9 class e *o dilm230-073-83-470-0 40dln macedo 2bruno miguel de pina a1339 sand lime rd winter garden, fl 34787-4291 3dob 12/30/1983 15 sex m 4b exp 12/30/202016 hgt 5-11" 12 rest none si safe driver 31d0b suns staf t83 9a end none 1de hesunghi a5/eive arriv tate 4a iss 12/12/2012 ine 5 dd g711906070623 oooinoooo arrivealive replaced 06/07/2019 suneh veteran m he state operation of a motor vehicle constitutes consent to any sobriety test required by law. donor 10000000 tri ': {'birth_date': '12/30/1983',
  'dl_class': 'a',
  'dl_number': 'm230-073-83-470-0',
  'expiration_date': '12/30/2020',
  'eyes': '',
  'sex': 'm'}}

### Georgia:

In [ ]:
def georgia_extractor(carteira):
    georgia_dl_info = dict()
    s = re.search(r'((?<=sex )([f|m]))', str(carteira))
    o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(carteira))
    exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    dl = re.search(r'(?<=dl no. )(\d{6,12})', str(carteira))
    n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    cl = re.search(r'(?<=class )([a|b|c|e|f|p|m])', str(carteira))
    #Setando o número mínimo de argumentos aceitos para 3:
    num = ( bool( s ) + bool( o ) + bool( n ) + bool( exp ) + bool( dl )  )
    #Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
    if num >= 2:
        georgia_dl_info[carteira] = dict(sex =  s.group() if not s is None else '',
                            eyes =  o.group() if not o is None else '',
                            expiration_date =  exp.group() if not exp is None else '',
                           dl_number =  dl.group() if not dl is None else '',
                            birth_date =  n.group() if not n is None else '',
                                         name = name_extractor(carteira),
                                         location = location_extractor(carteira),
                                      dl_class = cl.group() if not cl is None else '')
    
    return georgia_dl_info

In [ ]:
carteira_georgia = 'usa georgia driver\'s license porno bith 120863 cossioner dl no. 020718220 dob 12/08/1963 class c exp 12/08/2026 marcie t williams 492 bethlehem rd locust grove, ga 30248-2546 henry restrictions ab end none iss 01/15/2019 sex f eyes bro hgt 6-00" wgt 180 lb dd 369028917900049118 suares www usageorgiadriver\'slicensepornobith120863cossionerdlno.020718220dob12/08/1963classcexp12/08/2026marcietwilliams492bethlehemrdlocustgrove,ga30248-2546henryrestrictionsabendnoneiss01/15/2019sexfeyesbrohgt6-00"wgt180lbdd369028917900049118suareswww'
georgia_extractor(carteira_georgia)

### Utah:

In [ ]:
def utah_extractor(carteira):
    utah_dl_info = dict()
    s = re.search(r'((?<=sex )([f|m]))', str(carteira))
    o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(carteira))
    exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    dl = re.search(r'(\d{9})', str(carteira))
    #dl = re.search(r'(?<=(4d )|( - ))(\d{6,12})', str(carteira))
    n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(carteira))
    cl = re.search(r'(?<=class )([a|b|c|d|e|f|p|m])', str(carteira))
    #Setando o número mínimo de argumentos aceitos para 3:
    num = ( bool( s ) + bool( o ) + bool( n ) + bool( exp ) + bool( dl )  )
    #Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
    if num >= 2:
        utah_dl_info[carteira] = dict(sex =  s.group() if not s is None else '',
                            eyes =  o.group() if not o is None else '',
                            expiration_date =  exp.group() if not exp is None else '',
                           dl_number =  dl.group() if not dl is None else '',
                            birth_date =  n.group() if not n is None else '',
                                      name = name_extractor(carteira),
                                      location = location_extractor(carteira), 
                                      dl_class = cl.group() if not cl is None else '')
    
    return utah_dl_info

In [ ]:
carteira_utah =  'utah déronkerglase 158625619 ss 02/23/2018 v dob 02/25/1979. exp 02/25/2023 jensen kraig bert e 365 w 300 n salina, ut 846.54 do 3648351 class a nd t u sex m 12 rest a 16 hot 6-02 17 wgt 285 lb 18 eyes blu 0 2125/79 har bln g 02/25/79 utahdéronkerglase158625619ss02/23/2018vdob02/25/1979.exp02/25/2023jensenkraigberte365w300nsalina,ut846.54do3648351classandtusexm12resta16hot6-0217wgt285lb18eyesblu02125/79harblng02/25/79'
utah_extractor(carteira_utah)

### Function to decide which extractor to use:

In [ ]:
def decide_extract(carteira):
    # From the state_classifier output, decide which extractor to use:
    if state_classifier(carteira) == 'florida':
        return florida_extractor(carteira)
    
    if state_classifier(carteira) == 'georgia':
        return georgia_extractor(carteira)
    
    if state_classifier(carteira) == 'utah':
        return utah_extractor(carteira)

    

### Testing the main function:

In [ ]:
decide_extract(carteira_florida)

In [ ]:
decide_extract(carteira_utah)

In [ ]:
decide_extract(carteira_georgia)

### Name extractor

In [ ]:
nlp = spacy.load("en_core_web_md")
def name_extractor(carteira):
    name_list = list()
    doc = nlp(carteira)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            name_list.append(ent)
            
    return name_list

OSError: ignored

In [ ]:
name_extractor(carteira_utah)

In [ ]:
def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)
    person_list = []
    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []

    return (person_list)


names = get_human_names(filtered_text)
print("LAST, FIRST")
for name in names: 
    last_first = HumanName(name).last + ', ' + HumanName(name).first
    print(last_first)

In [ ]:
stopwords = ['florida', 'exp', 'dob', 'iss', 'eyes', 'driver',
             'license', 'class', 'state', 'motor', 'constitutes',
             'to', 'is', 'an', 'any', 'sex', 'usa', 'georgia', 'utah',
             'wgt', 'rest', 'blu', 'bro', 'brn', 'blk', 'hzl', 'end', 
             'none', 'replace', 'operation', 'exp', 'rest', 'hgt', 'bln',
             'usa', 'driver\'s', 'lb', 'dl no', 'dd', 'medical', 'information', 'governor',
             '/', ')', '(']

def filter_dl(carteira):
    for word in stopwords:
        if word in carteira:
            carteira = carteira.replace(" " + word + " ", " ")
    
    return carteira

In [ ]:
carteira_georgia = 'usa ( georgia driver\'s license governor moton dut 010382 dl no. 049938575 dob 01/03/1982 class c exp 01/03/2027 celetha qen thompson commissioner 273 plantation dr east point sa 30344-2272 fulton resulctions a end none iss 01/05/2019 sex f eyes bro hgt 5-06" wgt 172 lb dd 368039938510048651 althe hann urgent medical information on reverse donor usa(georgiadriver\'slicensegovernormotondut010382dlno.049938575dob01/03/1982classcexp01/03/2027celethaqenthompsoncommissioner273plantationdreastpointsa30344-2272fultonresulctionsaendnoneiss01/05/2019sexfeyesbrohgt5-06"wgt172lbdd368039938510048651althehannurgentmedicalinformationonreversedonor'
filtered = filter_dl(carteira_georgia)
filtered

In [ ]:
name_extractor(filtered)

In [ ]:
for word in stopwords:
    if word in carteira_utah:
        carteira_florida = carteira_florida.replace(" " + word + " ", " ")
carteira_florida

### Location extractor

In [ ]:

def location_extractor(carteira):
    doc = nlp(carteira)
    loc_list = list()
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            loc_list.append(ent.text)
    
    return loc_list

In [ ]:
location_extractor("p. florida usa grigat .. driver license 9 class e *o dilm230-073-83-470-0 40dln macedo 2bruno miguel de pina a1339 sand lime rd winter garden, fl 34787-4291 3dob 12/30/1983 15 sex m 4b exp 12/30/202016 hgt 5-11\" 12 rest none si safe driver 31d0b suns staf t83 9a end none 1de hesunghi a5/eive arriv tate 4a iss 12/12/2012 ine 5 dd g711906070623 oooinoooo arrivealive replaced 06/07/2019 suneh veteran m he state operation of a motor vehicle constitutes consent to any sobriety test required by law. donor 10000000 tri ")

In [ ]:
nlp = spacy.load("en_core_web_md")
doc = nlp("049938575 01/03/1982 c 01/03/2027 ")

for ent in doc.ents:
    print(ent)

In [ ]:
result = re.sub('(\d)|(-)|(/)',' ',carteira_florida) 
result

### Setting to OOP

In [ ]:
import re
import spacy

class driver_license:

    def __init__(self, text):
        self.text = text
        
        
        self.name_stopwords = ['florida', 'exp', 'dob', 'iss', 'eyes', 'driver',
                       'license', 'class', 'state', 'motor', 'constitutes',
                       'to', 'is', 'an', 'any', 'sex', 'usa', 'georgia', 'utah',
                       'wgt', 'rest', 'blu', 'bro', 'brn', 'blk', 'hzl', 'end',
                       'none', 'replace', 'operation', 'exp', 'rest', 'hgt', 'bln',
                       'usa', 'driver\'s', 'lb', 'dl no', 'dd', 'medical', 'information', 'governor',
                       '/', ')', '(', 'governory', 'comissioner', 'restriction', 'restrictions',
                       'donor', 'aclass', 'dln', 'oooinoooo', 'arrivealive', 'veteran']
        
        
    def state(self):

        florida_re = re.search(r' ?f?l?(orid)a? ?', str(self.text))
        dln_florida_re = re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', str(self.text))
        georgia_re = re.search(r' ?g?(eorgi)a? ?', str(self.text))
        utah_re = re.search(r' ?(utah) ?', str(self.text))
        
        if florida_re or dln_florida_re:
            self.state = 'florida'
        elif georgia_re:
            state = 'georgia'
        elif utah_re:
            self.state = 'utah'
        else: 
            self.state = 'outlier'

        return self.state
    
    
    def info(self):
    # From the state_classifier output, decide which extractor to use:
        if self.state == 'florida':
            
            self.info = dict()
            
            s = re.search(r'((?<=sex )([f|m]))', str(self.text))
            o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(self.text))
            exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            dl = re.search('(\D{1}\d{3}-\d{1,3}-\d{1,3}-\d{1,3}-\d{1})', str(self.text))
            n = re.search(r'(?<=[(dob )|(date of birth )] )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            cl = re.search(r'(?<=[9 |class ])([a|b|c|e|])', str(self.text))
            # Setando o número mínimo de argumentos aceitos para 3:
            num = (bool(s) + bool(o) + bool(n) + bool(exp) + bool(dl))
            # Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
            if num >= 2:
                
                self.info[self.text] = dict(sex=s.group() if not s is None else '',
                                                 eyes=o.group() if not o is None else '',
                                                 expiration_date=exp.group() if not exp is None else '',
                                                 dl_number=dl.group() if not dl is None else '',
                                                 birth_date=n.group() if not n is None else '',
                                                 name=self.name,
                                                 #location=location_extractor(carteira),
                                                 dl_class=cl.group() if not cl is None else '')

            return self.info

        elif self.state == 'georgia':
            self.info = dict()
            s = re.search(r'((?<=sex )([f|m]))', str(self.text))
            o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(self.text))
            exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            dl = re.search(r'(?<=(dl no. |dl no.)(\d{6,12})', str(self.text))
            n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            cl = re.search(r'(?<=class )([a|b|c|e|f|p|m])', str(self.text))
            # Setando o número mínimo de argumentos aceitos para 3:
            num = (bool(s) + bool(o) + bool(n) + bool(exp) + bool(dl))
            # Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
            if num >= 2:
                
                self.info[self.text] = dict(sex=s.group() if not s is None else '',
                                                 eyes=o.group() if not o is None else '',
                                                 expiration_date=exp.group() if not exp is None else '',
                                                 dl_number=dl.group() if not dl is None else '',
                                                 birth_date=n.group() if not n is None else '',
                                                 name=self.name,
                                                 #location=location_extractor(carteira),
                                                 dl_class=cl.group() if not cl is None else '')

            return self.info

        elif self.state == 'utah':
            
            self.info = dict()
            
            s = re.search(r'((?<=sex )([f|m]))', str(self.text))
            o = re.search(r' ?(blu)|(bro)|(blk)|(hzl)|(grn) ?', str(self.text))
            exp = re.search(r'(?<=exp )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            dl = re.search(r'(\d{9})', str(self.text))
            n = re.search(r'(?<=[dob |date of birth ] )(\d{1,2}/\d{1,2}/\d{1,4})', str(self.text))
            cl = re.search(r'(?<=class )([a|b|c|d|e|f|p|m])', str(self.text))
            # Setando o número mínimo de argumentos aceitos para 3:
            num = (bool(s) + bool(o) + bool(n) + bool(exp) + bool(dl))
            # Caso a quantia de argumentos aceitos seja igual/maior a 3, será adicionada a carteira no dicionario:
            if num >= 2:
                
                self.info[self.text] = dict(sex=s.group() if not s is None else '',
                                              eyes=o.group() if not o is None else '',
                                              expiration_date=exp.group() if not exp is None else '',
                                              dl_number=dl.group() if not dl is None else '',
                                              birth_date=n.group() if not n is None else '',
                                              name=self.name,
                                              #location=location_extractor(carteira),
                                              dl_class=cl.group() if not cl is None else '')
                
            
            return self.info
        
    def name(self):
        name_list = list()
        
        stopwords_names = ['florida', 'exp', 'dob', 'iss', 'eyes', 'driver',
                       'license', 'class', 'state', 'motor', 'constitutes',
                       'to', 'is', 'an', 'any', 'sex', 'usa', 'georgia', 'utah',
                       'wgt', 'rest', 'blu', 'bro', 'brn', 'blk', 'hzl', 'end',
                       'none', 'replace', 'operation', 'exp', 'rest', 'hgt', 'bln',
                       'usa', 'driver\'s', 'lb', 'dl no', 'dd', 'medical', 'information', 'governor',
                       '/', ')', '(', 'governory', 'comissioner', 'restriction', 'restrictions',
                       'donor', 'aclass', 'dln', 'oooinoooo', 'arrivealive', 'veteran']
        
        
        def filter_for_name_dl(carteira):
            
            for word in self.name_stopwords:
                
                if word in carteira:
                    carteira = carteira.replace(" " + word + " ", " ")
                    carteira = re.sub('(\d)|(-)|(/)', ' ', carteira)
                    
            return carteira
        
        nlp = spacy.load('en_core_web_md')
        doc = nlp(filter_for_name_dl(self.text))
        
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                name_list.append(ent.text)
        
        self.name = name_list
        
        
        return self.name
        
    

In [ ]:
dl = driver_license("p. florida usa grigat .. driver license 9 class e *o dilm230-073-83-470-0 40dln macedo 2bruno miguel de pina a1339 sand lime rd winter garden, fl 34787-4291 3dob 12/30/1983 15 sex m 4b exp 12/30/202016 hgt 5-11 12 rest none si safe driver 31d0b suns staf t83 9a end none 1de hesunghi a5/eive arriv tate 4a iss 12/12/2012 ine 5 dd g711906070623 oooinoooo arrivealive replaced 06/07/2019 suneh veteran m he state operation of a motor vehicle constitutes consent to any sobriety test required by law. donor 10000000 tri")
dl.info(), dl.name()